<a href="https://colab.research.google.com/github/Fennec2000GH/GeoGenie/blob/main/NLP_for_Geography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing for Geography

## Installations

Install Anaconda.

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
! chmod +x Miniconda3-py39_4.9.2-Linux-x86_64.sh
! bash ./Miniconda3-py39_4.9.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')

--2022-04-10 04:27:49--  https://repo.anaconda.com/miniconda/Miniconda3-py39_4.9.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61451533 (59M) [application/x-sh]
Saving to: ‘Miniconda3-py39_4.9.2-Linux-x86_64.sh’

Miniconda3-py39_4.9 100%[===================>]  58.60M   106MB/s    in 0.6s    

2022-04-10 04:27:50 (106 MB/s) - ‘Miniconda3-py39_4.9.2-Linux-x86_64.sh’ saved [61451533/61451533]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - brotlipy==0.7.0=py39h27cfd23_1003
    - ca-certificates==2020.12.8=h06a4308_0
    - certifi==2020.12.5=py39h06a4308_0
    - cffi==1.14.4=py39h261ae71_0
    - chardet==3.0.4=py39h06a4308_1003
   

In [ ]:
%%shell
ls /usr/local/
export PYTHONPATH=/usr/local/
echo $PYTHONPATH

bin		 cuda-11	     include	  share
compiler_compat  cuda-11.0	     lib	  shell
condabin	 cuda-11.1	     LICENSE.txt  src
conda-meta	 envs		     licensing	  ssl
cuda		 etc		     man	  x86_64-conda_cos6-linux-gnu
cuda-10.0	 games		     pkgs	  xgboost
cuda-10.1	 _gcs_config_ops.so  sbin
/usr/local/


Install Python packages for development.

In [111]:
%%shell
pip install -r requirements.txt

  Using cached tensorflow-2.8.0-cp39-cp39-manylinux2010_x86_64.whl (497.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 636.8/636.8 KB 1.3 MB/s eta 0:00:0000:0100:01
  Using cached numpy-1.22.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.22.3 which is incompatible.
cltk 1.0.23 requires PyYAML<6.0.0,>=5.4.1, but you have pyyaml 6.0 which is incompatible.
cltk 1.0.23 requires spacy<3.0.0,>=2.3.5, but you have spacy 3.2.4 which is incompatible.
bokeh 2.4.1 requires typing-extensions>=3.10.0, but you have typing-extens

## Load Dataset

In [5]:
import os
import numpy as np
import pandas as pd
import pyquery
import tqdm
from termcolor import cprint, colored

def preprocess_car_reviews():
    """Get TEXT portion of all XML files and assign each text a label of 0 for car reviews datasets."""
    df = pd.DataFrame(
        columns=np.asarray(a=list(["text", "label"])), 
        dtype=np.asarray(a=list([np.str, np.int64])),
    )

    for year in tqdm.tqdm(iterable=np.arange(2007, 2010)):
        cprint(text=year, color="yellow")
        for car_review_xml in tqdm.tqdm(iterable=os.listdir(path=f"./datasets/cars/{year}/")):
            try:
                abs_path = os.path.abspath(path=os.path.join(f"./datasets/cars/{year}/", car_review_xml))
                xml_content = open(file=abs_path).read()
                TEXT = pyquery.PyQuery(xml_content).find(selector="TEXT").text()
                # cprint(text=abs_path, color="blue")
                # cprint(text=xml_content, color="white")
                # cprint(text=TEXT, color="green")
                df = df.append(other=pd.DataFrame(
                        data=dict({
                            "text": " ".join(TEXT.split()[:512]), 
                            "label": 0
                        }),
                        index=[len(df)],
                    ))
            except:
                continue

    return df

def preprocess_wiktor():
    """Get TEXT portion of all XML files and assign each text a label of 1 for WikToR dataset."""
    df = pd.DataFrame(
            columns=np.asarray(a=list(["text", "label"])), 
            dtype=np.asarray(a=list([np.str, np.int64])),
    )

    abs_path = os.path.abspath(path="./datasets/WikToR.xml")
    xml_content = open(file=abs_path).read().encode(encoding="utf-8")
    # cprint(text=abs_path, color="blue")
    # cprint(text=xml_content[:1000], color="white")

    for text_node in tqdm.tqdm(iterable=pyquery.PyQuery(xml_content).find(selector="TEXT")):
        text = pyquery.PyQuery(text_node).text()
        # cprint(text=text, color="green")
        df = df.append(other=pd.DataFrame(
                data=dict({
                    "text": " ".join(text.split()[:512]), 
                    "label": 1
                }),
                index=[len(df)],
            ))

    return df

In [6]:
preprocess_datasets = dict({
    "car_reviews": preprocess_car_reviews,
    "wiktor": preprocess_wiktor,
})

Concat source datasets into one master dataset.

In [7]:
master_df = None
try:
    master_df = pd.read_csv(filepath_or_buffer="master_df.csv")
except:
    source_datasets = [preprocess_func() for preprocess_func in list(preprocess_datasets.values())]
    master_df = pd.concat(objs=source_datasets, ignore_index=True)
    master_df.to_csv(path_or_buf="master_df.csv")

print(master_df)

      Unnamed: 0                                               text  label
0              0  I just moved to Germany two months ago and bou...      0
1              1  Had for 1 year, 14,000 miles. Excellent fuel e...      0
2              2  I really love my Acura TL-Type S. The fit and ...      0
3              3  The TSX is really a very solid car. I was plea...      0
4              4  We got the A3 because it was fun to drive and ...      0
...          ...                                                ...    ...
5560        5560  Tahu is a village development committee in Pal...      1
5561        5561  Margate is a seaside town in the district of T...      1
5562        5562  Margate is a city in Broward County, Florida, ...      1
5563        5563  Margate is a small seaside town on the Channel...      1
5564        5564  Margate is a residential suburb of the Moreton...      1

[5565 rows x 3 columns]


## Generate Sentence Embeddings

In [9]:
import nltk
nltk.download(info_or_id="punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
%%shell
python -m textblob.download_corpora

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [37]:
import textblob as tb
from pprint import pprint
from typing import Iterable
from sentence_transformers import SentenceTransformer

def text_to_sent_embeddings_df(text: str, label: int, model_name_or_path: str): 
    model = SentenceTransformer(model_name_or_path=model_name_or_path)
    corpus = tb.TextBlob(text=text)
    sentences = np.asarray(a=list([str(sent.correct()) for sent in corpus.sentences]))
    # pprint(sentences)
    embeddings = model.encode(sentences=sentences)
    embedding_size = embeddings.shape[1]
    cprint(text=f"embedding_size: {embedding_size}", color="yellow")

    df = pd.DataFrame(
            columns=np.asarray(a=list(["embedding", "label"])), 
            dtype=np.asarray(a=list([Iterable[float], np.int64])),
        )

    for sentence, embedding in tqdm.tqdm(iterable=zip(sentences, embeddings)):
        # cprint(text=sentence, color="blue")
        # cprint(text=embedding, color="green")
        # print(type(embedding))
        # print(embedding.shape)
        df = df.append(other=pd.DataFrame(
                data=dict({
                    "embedding": (embedding, ),
                    "label": label
                }),
                index=[len(df)],
            ))

    return df

In [155]:
num_texts_for_sent_embeddings = 10
master_df_shuffled = master_df.sample(frac=1).iloc[:num_texts_for_sent_embeddings, :]
while len(master_df_shuffled.iloc[:num_texts_for_sent_embeddings, :][master_df_shuffled["label"] == 1]) == 0:
    master_df_shuffled = master_df.sample(frac=1)
master_df_shuffled.index = np.arange(num_texts_for_sent_embeddings)

print(master_df_shuffled)
sent_embedding_df_collection = [text_to_sent_embeddings_df(text=master_df_shuffled.loc[idx, "text"], label=master_df_shuffled.loc[idx, "label"], model_name_or_path="all-MiniLM-L6-v2") 
                               for idx in tqdm.trange(num_texts_for_sent_embeddings)]

   Unnamed: 0                                               text  label
0        1988  Fairfield is a town in Utah County, Utah, Unit...      1
1        4437  Kampala is the capital and largest city of Uga...      1
2        4111  Gedi is a village near Rapar of Kutch district...      1
3        1961  Sande is a municipality in Møre og Romsdal cou...      1
4        5010  Florencia (Spanish pronunciation: [floˈɾensja]...      1
5        4937  Clovis is the county seat of Curry County, New...      1
6         382  Astra owners, check to make sure your lights, ...      0
7        3995  Meta is a city in Osage County, Missouri, Unit...      1
8        3977  Greenwich is a suburb on the lower North Shore...      1
9         362  If I had to give up my wife of 20 years or NIS...      0







  0%|          | 0/10 [02:59<?, ?it/s]


embedding_size: 384



9it [00:00, 635.88it/s]







embedding_size: 384


...


13it [00:00, 443.26it/s]







embedding_size: 384



9it [00:00, 687.02it/s]







embedding_size: 384



28it [00:00, 465.42it/s]





100%|██████████| 10/10 [01:12<00:00,  7.27s/it]


In [159]:
import pickle

sent_embedding_master_df = None
try:
    # sent_embedding_master_df = pd.read_csv(filepath_or_buffer="sent_embedding_master_df.csv")
    sent_embedding_master_df = pickle.load(file=open(file="sent_embedding_master_df.pkl", mode="rb"))
except:
    sent_embedding_master_df = pd.concat(objs=sent_embedding_df_collection, ignore_index=True)
    # sent_embedding_master_df.to_csv(path_or_buf="sent_embedding_master_df.csv")
    pickle.dump(obj=sent_embedding_master_df, file=open(file="sent_embedding_master_df.pkl", mode="wb"))

print(sent_embedding_master_df)

                                             embedding label
0    [0.09670934, -0.037960503, 0.01115501, -0.0090...     1
1    [0.08566087, -0.023206292, 0.05576405, 0.05688...     1
2    [0.043077007, 0.034354337, 0.0046972157, -0.03...     1
3    [0.016493728, -0.051733445, 0.029489147, 0.039...     1
4    [0.04074373, 0.015332313, -0.06719875, 0.04199...     1
..                                                 ...   ...
141  [0.015216866, 0.018607464, 0.055264708, 0.0532...     0
142  [-0.018132692, 0.009903421, 0.019440347, 0.018...     0
143  [-0.033319887, -0.031448375, 0.07788969, 0.034...     0
144  [-0.027596964, 0.03516169, -0.0037372904, 0.02...     0
145  [0.021741537, -0.011845277, 0.010979724, 0.004...     0

[146 rows x 2 columns]


## AutoML

Setup H2O framework.

In [42]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/fennec2000/anaconda3/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_5rlc01d
  JVM stdout: /tmp/tmp_5rlc01d/h2o_root_started_from_python.out
  JVM stderr: /tmp/tmp_5rlc01d/h2o_root_started_from_python.err
huggingface/tokenizers: The current process just got forked, after parallelism has

H2O_cluster_uptime:,17 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.4
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_root_nag7om
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.743 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Convert sentence embedding dataframe to full-sized matrix.

In [162]:
embedding_size = sent_embedding_master_df.loc[0, "embedding"].size
print(embedding_size)

matrix = np.empty(shape=(len(sent_embedding_master_df), embedding_size + 1))
for idx in tqdm.tqdm(iterable=np.arange(len(sent_embedding_master_df))):
    matrix[idx, :embedding_size] = sent_embedding_master_df.loc[idx, "embedding"][:embedding_size]
    matrix[idx, -1] = sent_embedding_master_df.loc[idx, "label"]

print(matrix.shape)
print(matrix)

384



100%|██████████| 146/146 [00:00<00:00, 29280.31it/s]

(146, 385)
[[ 9.67093408e-02 -3.79605033e-02  1.11550102e-02 ...  1.46028129e-02
  -6.05137832e-02  1.00000000e+00]
 [ 8.56608674e-02 -2.32062917e-02  5.57640493e-02 ... -1.02798156e-02
  -1.56932196e-03  1.00000000e+00]
 [ 4.30770069e-02  3.43543366e-02  4.69721574e-03 ... -8.21788907e-02
  -1.44447237e-02  1.00000000e+00]
 ...
 [-3.33198868e-02 -3.14483754e-02  7.78896883e-02 ... -7.12964742e-04
  -1.84733197e-02  0.00000000e+00]
 [-2.75969636e-02  3.51616889e-02 -3.73729039e-03 ...  1.95916481e-02
   4.54430208e-02  0.00000000e+00]
 [ 2.17415374e-02 -1.18452767e-02  1.09797241e-02 ...  3.39486748e-02
   4.37198430e-02  0.00000000e+00]]


Train, Test, Split.

In [167]:
h2o_df = h2o.H2OFrame(python_obj=matrix)
h2o_df.set_names(names=[str(idx) for idx in np.arange(embedding_size)] + ["label"])
h2o_df["label"] = h2o_df["label"].asfactor()
print(h2o_df)
train, test, valid = h2o_df.split_frame(ratios=[0.70, 0.15])

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,label
0.0967093,-0.0379605,0.011155,-0.00901961,-0.0632418,-0.00586466,-0.0324938,-0.0870255,-0.0290042,0.0239152,0.0246682,0.00328497,0.156851,0.0192161,-0.0404984,0.0178701,0.0214358,0.0635899,-0.0210897,0.0699571,0.0408291,0.0698819,-0.0255885,0.0450414,-0.0296469,-0.0146104,0.00373326,-0.0973911,-0.0520566,-0.00664738,-0.074376,-0.0572518,0.00288363,-0.0703388,-0.0168602,-0.0127628,0.0501417,0.0465486,0.0173382,-0.00876422,-0.0477691,0.0283454,0.0195712,-0.0298233,-0.00517097,0.0782555,0.0702206,-0.141423,0.0920922,0.0112023,0.0507247,0.0263916,0.0203618,0.0193318,-0.0306278,0.0210263,-0.101227,0.00347226,0.026032,-0.05794,0.0312215,-0.0104209,-0.0294705,-0.0313026,-0.0235906,0.0204308,0.0780687,-0.0187966,0.0761966,-0.12777,0.0365432,0.06188,0.0142166,-0.0203261,-0.0332429,0.0509202,0.0334272,0.0380012,0.0108791,0.0193318,0.00337154,-0.061425,0.00206747,0.0494716,-0.000430733,0.0210239,0.0188068,0.0620451,0.110145,0.0183684,-0.0442104,-0.0373742,-0.010877,-0.0346136,-0.0190488,-0.103733,-0.0879044,0.0310848,-0.0299158,0.0342269,0.026929,0.0351522,0.0389501,-0.0277862,-0.0165969,0.0667126,-0.0456127,0.0664288,-0.00793672,-0.018094,0.0622096,0.00499585,-0.00725015,-0.0511326,-0.00185325,0.14302,0.0434493,-0.064407,-0.0245494,0.00910663,0.0267761,0.00936751,0.0578444,0.0278609,0.045055,-0.0544278,-0.0434005,-5.13805e-33,-0.0699341,0.0459919,0.0671142,0.00397329,-0.00125836,0.000728814,0.0307257,0.0051345,-0.0547455,-0.0665734,0.037243,0.0199907,0.0229834,-0.00172012,-0.0290951,0.103737,0.00952084,-0.0635133,-0.0525373,-0.0210135,-0.0755548,-0.00507576,-0.0381875,0.0960627,-0.0574455,-0.0411107,-0.00129896,0.00648406,0.00150575,-0.00539232,0.0679188,-0.0237284,0.0355806,0.101205,0.103374,0.059641,-0.00490703,0.0572433,-0.00548148,0.0786111,-0.00589569,-0.0363783,0.00997938,0.0864676,-0.0348673,-0.0475564,0.00930545,0.0116355,-0.096731,-0.0788226,0.0498521,0.040075,-0.0592877,0.0261773,-0.0234682,-0.092161,-0.00333375,-0.0372823,-0.0183521,0.0577903,0.0633256,0.0311819,-0.0805374,0.081326,-0.0923395,-0.0307361,-0.0315426,-0.0170594,0.0290845,-0.0119024,0.0860515,0.0154925,0.0343003,0.0680273,-0.00187899,-0.0122595,-0.0103718,-0.0593081,-0.0336412,0.0144635,0.0311841,-0.115982,-0.124082,0.0214798,-0.105545,0.0637127,0.0882656,-0.0773576,-0.00326556,0.00976479,-0.0315923,0.0789976,0.131128,-0.00725434,0.0295019,1.75411e-33,-0.0406945,-0.0248579,-0.00767201,-0.00303182,-0.0224497,-0.0312232,0.0459585,-0.0679439,-0.0225089,-0.105581,0.0176062,-0.077868,-0.0614237,0.0727424,0.036102,0.0264183,0.0468658,-0.0547287,0.0348678,-0.0267989

In [168]:
from h2o.automl import H2OAutoML
aml = H2OAutoML(max_models=10, max_runtime_secs=100, seed=1)
aml.train(x=[str(idx) for idx in np.arange(embedding_size)], y = "label", training_frame=train)

AutoML progress: |█████
03:59:52.158: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 101.0.

██████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_2_AutoML_9_20220413_35943

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 3.180448980629709e-08
RMSE: 0.00017833813334869547
LogLoss: 8.394694907977935e-05
Null degrees of freedom: 100
Residual degrees of freedom: 98
Null deviance: 137.77974522890338
Residual deviance: 0.016957283714117494
AIC: 6.016957283714118
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9991987646846168: 


,,0,1,Error,Rate
0,0,43.0,0.0,0.0,(0.0/43.0)
1,1,0.0,58.0,0.0,(0.0/58.0)
2,Total,43.0,58.0,0.0,(0.0/101.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,9.991988e-01,1.000000,57.0
1,max f2,9.991988e-01,1.000000,57.0
2,max f0point5,9.991988e-01,1.000000,57.0
3,max accuracy,9.991988e-01,1.000000,57.0
4,max precision,9.999999e-01,1.000000,0.0
5,max recall,9.991988e-01,1.000000,57.0
6,max specificity,9.999999e-01,1.000000,0.0
7,max absolute_mcc,9.991988e-01,1.000000,57.0
8,max min_per_class_accuracy,9.991988e-01,1.000000,57.0
9,max mean_per_class_accuracy,9.991988e-01,1.000000,57.0



Gains/Lift Table: Avg response rate: 57.43 %, avg score: 57.42 %


...

,,0,1,Error,Rate
0,0,43.0,0.0,0.0,(0.0/43.0)
1,1,0.0,58.0,0.0,(0.0/58.0)
2,Total,43.0,58.0,0.0,(0.0/101.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,9.350632e-01,1.000000,57.0
1,max f2,9.350632e-01,1.000000,57.0
2,max f0point5,9.350632e-01,1.000000,57.0
3,max accuracy,9.350632e-01,1.000000,57.0
4,max precision,9.999998e-01,1.000000,0.0
5,max recall,9.350632e-01,1.000000,57.0
6,max specificity,9.999998e-01,1.000000,0.0
7,max absolute_mcc,9.350632e-01,1.000000,57.0
8,max min_per_class_accuracy,9.350632e-01,1.000000,57.0
9,max mean_per_class_accuracy,9.350632e-01,1.000000,57.0



Gains/Lift Table: Avg response rate: 57.43 %, avg score: 57.85 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.019802,9.999977e-01,1.741379,1.741379,1.0,9.999987e-01,1.000000,0.999999,0.034483,0.034483,74.137931,74.137931,0.034483
1,2,0.029703,9.999941e-01,1.741379,1.741379,1.0,9.999941e-01,1.000000,0.999997,0.017241,0.051724,74.137931,74.137931,0.051724
2,3,0.039604,9.999909e-01,1.741379,1.741379,1.0,9.999909e-01,1.000000,0.999996,0.017241,0.068966,74.137931,74.137931,0.068966
3,4,0.049505,9.999908e-01,1.741379,1.741379,1.0,9.999908e-01,1.000000,0.999995,0.017241,0.086207,74.137931,74.137931,0.086207
4,5,0.059406,9.999896e-01,1.741379,1.741379,1.0,9.999896e-01,1.000000,0.999994,0.017241,0.103448,74.137931,74.137931,0.103448
5,6,0.108911,9.999734e-01,1.741379,1.741379,1.0,9.999807e-01,1.000000,0.999988,0.086207,0.189655,74.137931,74.137931,0.189655
6,7,0.158416,9.999551e-01,1.741379,1.741379,1.0,9.999640e-01,1.000000,0.999980,0.086207,0.275862,74.137931,74.137931,0.275862
7,8,0.207921,9.998961e-01,1.741379,1.741379,1.0,9.999202e-01,1.000000,0.999966,0.086207,0.362069,74.137931,74.137931,0.362069
8,9,0.306931,9.994824e-01,1.741379,1.741379,1.0,9.996389e-01,1.000000,0.999861,0.172414,0.534483,74.137931,74.137931,0.534483
9,10,0.405941,9.972917e-01,1.741379,1.741379,1.0,9.981919e-01,1.000000,0.999454,0.172414,0.706897,74.137931,74.137931,0.706897


In [172]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_2_AutoML_9_20220413_35943,1,0.0169027,1,0,0.0737106,0.00543325
StackedEnsemble_AllModels_5_AutoML_9_20220413_35943,1,0.0379324,1,0,0.0873293,0.00762641
StackedEnsemble_BestOfFamily_6_AutoML_9_20220413_35943,1,0.0252058,1,0,0.0767418,0.0058893
StackedEnsemble_BestOfFamily_3_AutoML_9_20220413_35943,1,0.0230596,1,0,0.0843009,0.00710665
StackedEnsemble_BestOfFamily_1_AutoML_9_20220413_35943,1,0.0153023,1,0,0.068238,0.00465642
StackedEnsemble_AllModels_2_AutoML_9_20220413_35943,1,0.134085,1,0,0.152773,0.0233397
GLM_1_AutoML_9_20220413_35943,1,0.0334734,1,0,0.091817,0.00843036
StackedEnsemble_AllModels_1_AutoML_9_20220413_35943,1,0.133312,1,0,0.152742,0.02333
StackedEnsemble_BestOfFamily_4_AutoML_9_20220413_35943,1,0.0592514,1,0,0.0996099,0.00992213
StackedEnsemble_BestOfFamily_5_AutoML_9_20220413_35943,0.996592,0.0609743,0.997216,0.0116279,0.0992658,0.00985371
